# Exploring 215 records that hit on the Initial RegEx: 'XX NM dir XXX', but did not link to an FAA Airport Identifier
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 4, 2021
***
`This Jupyter Notebook explores 215 records (of the 5167 non-duplicate, standard format records) that hit on the initial 'XX NM dir XXX' regular expression, that do NOT link to the Airports Cleaned supplemental dataset by the IDENT field. The process for exploring these records is as follows:`

1. Grab only null values from dataframe containing 3025 records (these records did not link to the Airports Cleaned dataset on the 'IDENT' field).
<br>
2. Create a dictionary relating bearing abbreviations to their respective degrees.
<br>
3. Separate the UAS location into the distance (NM), bearing (abbrev), and NAVAID identifier.
<br>
4. Convert all distances from NM to kilometers, and all bearing abbreviations into their respective degrees. We also want to remove all records for which there is no bearing or distance information.
<br>
5. Calculate the UAS lat/long using the geopy library.
***
### INPUT: 
*Dataframe (read from a .csv file) containing 3025 records, of 5191 records that hit on the initial 'XX NM dir XXX' regular expression.*

### OUTPUT:
<font color='red'>**EXCEPTIONS:**</font>
*Runways: 5 records*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*NAVAID Systems Identifier: 177 records*

## Import Statements and Read Files

In [1]:
# Import Statements
import pandas as pd
import numpy as np

import geopy
from geopy.distance import geodesic

In [2]:
df1 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/std_format_uas_loc.csv')

In [3]:
df1

,REMARKS,UAS_LOC,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,2A962A82-442A-4A82-9535-214B1B978E2E,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE
1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC
2,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,50CC510F-CE80-405D-BE8D-05AFC123107F,San Diego Intl,32.733556,-117.189667,KSAN
3,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,28DD4E4B-2AD4-4122-AD63-A0842EB23993,Exec,28.545462,-81.332930,KORL
4,Aircraft observed a red quad copter UAS while ...,['4NM SSE of MLB'],MLB,3ACDEDBD-58E6-4FFF-A8A4-C7CAB20B3299,Melbourne Orlando Intl,28.102750,-80.645250,KMLB
...,...,...,...,...,...,...,...,...
3020,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,3646423B-964B-466A-9CE9-3CB43457F772,Addison,32.968556,-96.836444,KADS
3021,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,18FAF388-FEDD-48FC-88CF-83EA626E928D,Morristown Muni,40.799338,-74.414889,KMMU
3022,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,4285C89D-6393-48C3-B0AD-945171391D35,Buckeye Muni,33.420417,-112.686181,KBXK
3023,From MOR: Aircraft reported a NMAC with a flat...,['5 NM ENE of DRK'],DRK,NaN,NaN,NaN,NaN,NaN


#### There are 215 Records within the Standard Format Remarks with no Airport Information

In [4]:
df2 = df1[df1['NAME'].isnull()]
standard_no_airport_df = pd.DataFrame()
standard_no_airport_df['REMARKS'] = df2['REMARKS']
standard_no_airport_df['UAS_LOC'] = df2['UAS_LOC']
standard_no_airport_df['IDENT'] = df2['IDENT']

standard_no_airport_df

,REMARKS,UAS_LOC,IDENT
35,Phoenix PD advised a concerned citizen notifie...,['4NM W of RWY'],RWY
41,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL
56,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI
61,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ
99,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG
...,...,...,...
2969,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC
3007,"Aircraft reported a blue UAS, while NE bound a...",['1 NM NE of RWY'],RWY
3008,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH
3014,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ


### Import NAVAID Systems Cleaned Dataset

In [5]:
nav_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')

In [6]:
nav_clean

,LONGITUDE,LATITUDE,IDENT,NAME_TXT,CITY
0,-176.676024,51.871896,ADK,MOUNT MOFFETT,ADAK ISLAND
1,-151.744257,68.136626,AKP,ANAKTUVUK PASS,ANAKTUVUK PASS
2,-149.960164,61.167868,TED,ANCHORAGE,ANCHORAGE
3,-149.769204,61.255004,EDF,ELMENDORF,ANCHORAGE
4,-159.597931,61.590169,ANI,ANIAK,ANIAK
...,...,...,...,...,...
1669,-156.788152,71.273426,BRW,BARROW,UTQIAGVK
1670,-156.781302,71.282300,VIR,BROWERVILLE,UTQIAGVK
1671,-75.164524,43.026523,UCA,UTICA,UTICA
1672,-90.434470,40.937351,GBG,GALESBURG,GALESBURG


#### Join NAVAID Cleaned Dataset with standard_no_airport_df

In [7]:
# DATAFRAMES USED:
# nav_clean = NAVAID_cleaned.csv (the cleaned NAVAID_Systems dataset)
# standard_no_airport_df = dataframe of remarks in standard format with no airport information

uas_nav_loc = pd.merge(standard_no_airport_df, nav_clean, on='IDENT', how='left')

In [8]:
uas_nav_loc

,REMARKS,UAS_LOC,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,Phoenix PD advised a concerned citizen notifie...,['4NM W of RWY'],RWY,NaN,NaN,NaN,NaN
1,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,-74.159729,40.311641,COLTS NECK,COLTS NECK
2,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,-73.894446,40.612480,CANARSIE,CANARSIE
3,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,-74.741796,40.583053,SOLBERG,SOLBERG
4,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,-122.978171,45.353234,NEWBERG,NEWBERG
...,...,...,...,...,...,...,...
210,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,-80.211549,40.824941,ELLWOOD CITY,ELLWOOD CITY
211,"Aircraft reported a blue UAS, while NE bound a...",['1 NM NE of RWY'],RWY,NaN,NaN,NaN,NaN
212,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,-111.981924,40.850256,WASATCH,SALT LAKE CITY
213,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,-117.530009,33.918342,PARADISE,ONTARIO


In [9]:
len(uas_nav_loc[uas_nav_loc['NAME_TXT'].isnull()]) # THERE ARE 36 RECORDS THAT ARE NOT COVERED BY EITHER THE AIRPORTS_CLEANED OR NAVAID_CLEANED DATASETS


36

In [10]:
# Create dataframe of Remarks with Standard Format that do not contain airport or NAVAID references
uas_loc_noAir_noNav = uas_nav_loc[uas_nav_loc['NAME_TXT'].isnull()]

In [11]:
# Export to .csv file
uas_loc_noAir_noNav.to_csv('std_uasLoc_noAir_noNav.csv', index = False)

`We need to obtain the 5 runway records that we were not able to calculate UAS lat/long for`

In [34]:
rwy9 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/runways9_UAS_latlong.csv')
rwy9

,level_0,index,REMARKS,UAS_Location_Runways,IDENT,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,properties.GLOBAL_ID,...,Runway_Longitude,Runway_Latitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK,UAS_Latitude,UAS_Longitude
0,0,3,Aircraft observed a black UAS off the while E ...,['5NM W of RWY 08'],ATL,5.0,W,9.260,270,2A34B5BE-BB33-466C-89F6-7C1ED02AF028,...,-84.421287,33.649587,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL,8,True,33.649546,-84.521112
1,2,8,Aircraft observed a blue and white UAS over th...,['.5 NM S of RWY 19'],RVS,0.5,S,0.926,180,DC1F72BE-0DCB-4421-B016-1A01F4CC6EBC,...,-95.986255,36.038465,Richard Lloyd Jones Jr,36.039584,-95.984595,KRVS,19,True,36.030120,-95.986255
2,4,13,Aircraft observed a UAS 1 NM SW of RWY 26L app...,['1 NM SW of RWY 26'],LAS,1.0,SW,1.852,225,7A7E022E-83D3-4567-A82F-B88FA24C9CD1,...,-115.150560,36.076347,Harry Reid Intl,36.080044,-115.152235,KLAS,26,True,36.064544,-115.165096
3,6,15,Aircraft observed a UAS off the left side whil...,['1 NM W of RWY 25'],DVT,1.0,W,1.852,270,485921F4-43B5-405D-995A-DC3F2945CE77,...,-112.083559,33.689489,Phoenix Deer Valley,33.688306,-112.082556,KDVT,25,True,33.689487,-112.103533
4,8,20,Aircraft observed a UAS off the left side whil...,['2 NM W of RWY 26'],POC,2.0,W,3.704,270,3002D569-9E04-44B8-BBD5-B8A978C6269F,...,-117.781996,34.092129,Brackett Fld,34.091667,-117.781778,KPOC,26,True,34.092122,-117.822132
5,10,24,Aircraft observed a UAS overhead while N bound...,['2 NM S of RWY 35'],DFW,2.0,S,3.704,180,A088B20F-FBE9-4C4F-9937-F7ABADEE30CC,...,-97.026110,32.900981,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,35,True,32.867583,-97.026110
6,13,30,"Aircraft reported a UAS with red lights at 2,3...",['6 NM S of RWY 35'],END,6.0,S,11.112,180,CA1DE5CD-69D2-4753-BB5F-4B39534E9F93,...,-97.914727,36.345432,Vance AFB,36.339806,-97.917239,KEND,35,True,36.245292,-97.914727
7,16,37,Aircraft reported a quad copter UAS while 3NM ...,['3NM SW of RWY 31'],HIO,3.0,SW,5.556,225,8C9A0924-3F4B-421A-8A49-83AAD6CCE7EE,...,-122.953230,45.546832,Portland-Hillsboro,45.542122,-122.950722,KHIO,31,True,45.511472,-123.003506
8,18,40,"Aircraft reported a blue UAS, while NE bound a...",['1 NM NE of RWY 4'],FFZ,1.0,NE,1.852,45,32734985-68C4-4D56-BEC2-FD67F855D3A8,...,-111.728359,33.462219,Falcon Fld,33.460841,-111.728325,KFFZ,4,True,33.474025,-111.714270


In [37]:
rwy14 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/runways14.csv')
rwy14

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Phoenix PD advised a concerned citizen notifie...,"['RWY8', 'RWY8']",NaN,NaN,NaN,['4NM']
1,1,Aircraft observed a black UAS off the while E ...,['RWY 08'],NaN,NaN,NaN,['5NM']
2,2,Aircraft observed a blue and white UAS over th...,['RWY 19R'],.5 NM S of RVS,RVS,RWY 19R,['.5 NM']
3,3,Aircraft observed a UAS 1 NM SW of RWY 26L app...,['RWY 26L'],1 NM SW of LAS,LAS,RWY 26L,['1 NM']
4,4,Aircraft observed a UAS off the left side whil...,['RWY 25L'],1 NM W of DVT,DVT,RWY 25L,['1 NM']
5,5,0340 INITIAL/CLOSEOUT: Law Enforcement Aircraf...,['RWY8'],NaN,NaN,NaN,['1.5 NM']
6,6,Aircraft observed a UAS off the left side whil...,['RWY 26L'],1/2 NM W of POC,POC,RWY 26L,['2 NM']
7,7,Aircraft observed a UAS overhead while N bound...,['RWY 35C'],1/2 NM S of DFW,DFW,RWY 35C,['2 NM']
8,8,Aircraft observed a UAS off the right side whi...,['RWY 9'],NaN,NaN,NaN,['1 NM']
9,9,"Aircraft reported a UAS with red lights at 2,3...",['RWY 35C'],6 NM S of END,END,RWY 35C,['6 NM']


In [39]:
rwy5 = pd.merge(rwy14,rwy9, on = 'REMARKS', how='left', indicator=True)
rwy5_exp = rwy5[rwy5['_merge'] == 'left_only'].drop_duplicates(subset=['REMARKS']).reset_index(drop=True)
rwy5_exp

,index_x,REMARKS,UAS_Location_Runways_x,UASLOCATION,IDENT_x,RWYLOCATION,UAS_Distance,level_0,index_y,UAS_Location_Runways_y,...,Runway_Latitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK,UAS_Latitude,UAS_Longitude,_merge
0,0,Phoenix PD advised a concerned citizen notifie...,"['RWY8', 'RWY8']",NaN,NaN,NaN,['4NM'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,5,0340 INITIAL/CLOSEOUT: Law Enforcement Aircraf...,['RWY8'],NaN,NaN,NaN,['1.5 NM'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,8,Aircraft observed a UAS off the right side whi...,['RWY 9'],NaN,NaN,NaN,['1 NM'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,10,Aircraft reported a UAS 200 feet above their A...,['RWY20R'],NaN,NaN,NaN,['1NM'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,12,Aircraft reported a small UAS 4 NM SW of RWY19...,['RWY19R'],NaN,NaN,NaN,['4 NM'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [40]:
# Export to .csv file
rwy5_exp.to_csv('rwy5OF14_EXCEPTIONS.csv', index = False)

#### Of the 215 records following the standard format that do not have related Airport information, there are 36 that also do not have associated NAVAID information

These 36 records are currently removed from the dataframe

In [12]:
uas_nav_loc = uas_nav_loc.dropna().reset_index() # DROP ALL NULL VALUES FOR THE TIME BEING (LEAVES 2818 RECORDS)
uas_nav_loc

,index,REMARKS,UAS_LOC,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,1,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,-74.159729,40.311641,COLTS NECK,COLTS NECK
1,2,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,-73.894446,40.612480,CANARSIE,CANARSIE
2,3,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,-74.741796,40.583053,SOLBERG,SOLBERG
3,4,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,-122.978171,45.353234,NEWBERG,NEWBERG
4,5,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,-74.495028,40.202410,ROBBINSVILLE,ROBBINSVILLE
...,...,...,...,...,...,...,...,...
174,209,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,-75.303024,39.636039,WOODSTOWN,WOODSTOWN
175,210,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,-80.211549,40.824941,ELLWOOD CITY,ELLWOOD CITY
176,212,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,-111.981924,40.850256,WASATCH,SALT LAKE CITY
177,213,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,-117.530009,33.918342,PARADISE,ONTARIO


## Create Dictionary of Bearings and Respective Degrees

In [13]:
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

#### Break Up UAS_LOC column into the Distances and Bearing Abbreviations 

In [14]:
# Extract Distances and Bearings from UAS_LOC that were pulled from Remarks
# with a Standard Format in SkyWatch
uas_loc = uas_nav_loc['UAS_LOC']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(float(distance_nm[2:]))
    bearings.append(bearing_abrev)

In [15]:
# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_nav_loc['REMARKS'])
new_df['UAS_LOC'] = uas_nav_loc['UAS_LOC']
new_df['IDENT'] = uas_nav_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,10.0,NE
1,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,1.0,NE
2,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,8.0,N
3,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,20.0,WNW
4,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,10.0,NNE
...,...,...,...,...,...
174,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,8.0,SW
175,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,30.0,NW
176,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,14.0,S
177,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,1.0,SE


#### Convert all Distances from NM to Kilometers and Bearing Abbreviations into Degrees

In [16]:
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['NAVAID_Latitude'] = df1['LATITUDE']
new_df['NAVAID_Longitude'] = df1['LONGITUDE']

In [17]:
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude
0,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,10.0,NE,18.520,45,41.409407,-81.854691
1,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,1.0,NE,1.852,45,40.788388,-111.977773
2,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,8.0,N,14.816,0,32.733556,-117.189667
3,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,20.0,WNW,37.040,293,28.545462,-81.332930
4,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,10.0,NNE,18.520,23,28.102750,-80.645250
...,...,...,...,...,...,...,...,...,...
174,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,8.0,SW,14.816,225,42.542944,-83.177889
175,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,30.0,NW,55.560,315,40.777250,-73.872611
176,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,14.0,S,25.928,180,41.785972,-87.752417
177,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,1.0,SE,1.852,135,29.984435,-95.341442


In [18]:
# Export to .csv file
new_df.to_csv('std_uasLoc_navaid.csv', index = False)

#### Remove records from DataFrame with no Bearing Information Extracted

177 UAS sighting records remain

In [19]:
new_df['Bearing'].replace('', np.nan, inplace = True)
new_df.dropna(subset = ['Bearing'], inplace = True) #LEAVES 177 ROWS

new_df = new_df.reset_index()

In [20]:
new_df

,index,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude
0,0,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,10.0,NE,18.520,45,41.409407,-81.854691
1,1,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,1.0,NE,1.852,45,40.788388,-111.977773
2,2,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,8.0,N,14.816,0,32.733556,-117.189667
3,3,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,20.0,WNW,37.040,293,28.545462,-81.332930
4,4,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,10.0,NNE,18.520,23,28.102750,-80.645250
...,...,...,...,...,...,...,...,...,...,...
172,174,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,8.0,SW,14.816,225,42.542944,-83.177889
173,175,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,30.0,NW,55.560,315,40.777250,-73.872611
174,176,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,14.0,S,25.928,180,41.785972,-87.752417
175,177,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,1.0,SE,1.852,135,29.984435,-95.341442


In [21]:
new_df2 = pd.merge(new_df, nav_clean, on='IDENT', how='left')

In [22]:
new_df2 = new_df2.drop(['NAVAID_Latitude', 'NAVAID_Longitude'], 1)

In [23]:
new_df2 = new_df2.rename({'LONGITUDE': 'NAVAID_Longitude', 'LATITUDE': 'NAVAID_Latitude'}, axis=1)

In [24]:
new_df2

,index,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAVAID_Longitude,NAVAID_Latitude,NAME_TXT,CITY
0,0,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,10.0,NE,18.520,45,-74.159729,40.311641,COLTS NECK,COLTS NECK
1,1,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,1.0,NE,1.852,45,-73.894446,40.612480,CANARSIE,CANARSIE
2,2,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,8.0,N,14.816,0,-74.741796,40.583053,SOLBERG,SOLBERG
3,3,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,20.0,WNW,37.040,293,-122.978171,45.353234,NEWBERG,NEWBERG
4,4,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,10.0,NNE,18.520,23,-74.495028,40.202410,ROBBINSVILLE,ROBBINSVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...
172,174,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,8.0,SW,14.816,225,-75.303024,39.636039,WOODSTOWN,WOODSTOWN
173,175,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,30.0,NW,55.560,315,-80.211549,40.824941,ELLWOOD CITY,ELLWOOD CITY
174,176,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,14.0,S,25.928,180,-111.981924,40.850256,WASATCH,SALT LAKE CITY
175,177,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,1.0,SE,1.852,135,-117.530009,33.918342,PARADISE,ONTARIO


#### Calculate Lat/Long Coordinates for Sighted UAS using geopy

In [25]:
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_navaid = pd.to_numeric(new_df2['NAVAID_Latitude'][i])
    long_navaid = pd.to_numeric(new_df2['NAVAID_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_navaid, long_navaid)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

In [26]:
# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long

In [27]:
new_df2

,index,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAVAID_Longitude,NAVAID_Latitude,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,0,"Aircraft observed a UAS while NE bound at 3,00...",['10 NM NE of COL'],COL,10.0,NE,18.520,45,-74.159729,40.311641,COLTS NECK,COLTS NECK,40.429472,-74.005402
1,1,Aircraft observed a red UAS off the right wing...,['1 NM NE of CRI'],CRI,1.0,NE,1.852,45,-73.894446,40.612480,CANARSIE,CANARSIE,40.624272,-73.878968
2,2,Aircraft observed a UAS 8 NM N of SBJ VOR. No ...,['8 NM N of SBJ'],SBJ,8.0,N,14.816,0,-74.741796,40.583053,SOLBERG,SOLBERG,40.716474,-74.741796
3,3,Aircraft observed a gold colored UAS off the r...,['20 NM WNW of UBG'],UBG,20.0,WNW,37.040,293,-122.978171,45.353234,NEWBERG,NEWBERG,45.482623,-123.414277
4,4,Aircraft observed a black and red UAS heading ...,['10 NM NNE ofRBV'],RBV,10.0,NNE,18.520,23,-74.495028,40.202410,ROBBINSVILLE,ROBBINSVILLE,40.355907,-74.409843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,174,Aircraft reported a red UAS from theright side...,['8NM SWof OOD'],OOD,8.0,SW,14.816,225,-75.303024,39.636039,WOODSTOWN,WOODSTOWN,39.541614,-75.424897
173,175,"Aircraft reported a small blue UAS, pass appro...",['30 NM NW of EWC'],EWC,30.0,NW,55.560,315,-80.211549,40.824941,ELLWOOD CITY,ELLWOOD CITY,41.177757,-80.679758
174,176,"Aircraft reported a UAS at co-altitude of 9,00...",['14 NM S of TCH'],TCH,14.0,S,25.928,180,-111.981924,40.850256,WASATCH,SALT LAKE CITY,40.616773,-111.981924
175,177,Aircraft reported a NMAC with a silver UAS whi...,['1 NM SE of PDZ'],PDZ,1.0,SE,1.852,135,-117.530009,33.918342,PARADISE,ONTARIO,33.906535,-117.515850


In [28]:
# Export to .csv file
new_df2.to_csv('uas_latlong_standard_navaid_177.csv', index = False)